# Biome Level statistics for model: WK1

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [4]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage = ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Load the biomass density maps

In [5]:
# load the carbon density layers
potentialDensity = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK1_Potential_density_Ensambled_Mean").unmask()
presentDensity =  ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km").unmask()

# define the standard projection
stdProj = potentialDensity.projection();

## 3 Adjust the present and potential density maps

In [6]:
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask= presentForestCover.gt(0)
potentialMask= potentialCoverAdjusted.gt(0)

# check the difference of the two density maps
potentialHigher = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).gte(0)
potentialLower = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).lt(0)
# replace the lower potential value by present biomass density value
agbPotentialDensity = presentDensity.multiply(potentialLower).add(potentialDensity.multiply(potentialHigher))
# add the root biomass to the AGB to get TGB
tgbPotentialDensity = agbPotentialDensity.multiply(rootShootRatio).add(agbPotentialDensity)
tgbPresentDensity = presentDensity.multiply(rootShootRatio).add(presentDensity)

## 4 Partioning the potential cover into different landuse types

In [7]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [8]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = tgbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = tgbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

# calculate the part of the potential inside the forest cover which was allocate to conservation potential.
freelandForConservation1 = realDensityIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal))
freelandForConservation = freelandForConservation1.add(conservationPotentialPart2).rename('FreeToConservation')

absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

## Calculate the potential numbers and write into local folder

In [11]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/WK1_Biome_Level_Statistics.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,228.703254,257.437364,281.168861,4.704599,0.075468,8.042988,10.590268,0.318174,6.564083
1,9.864075,12.324969,20.880086,2.126831,0.019516,1.942910,4.411112,0.054748,0.732209
2,3.453565,4.919131,6.143256,0.444280,0.030265,0.437966,0.307579,0.004033,0.500517
3,49.234418,56.830716,75.578369,4.524483,0.024942,5.382451,8.354092,0.461686,2.537341
4,24.401686,26.739961,29.588381,1.313104,0.040701,0.841084,0.601932,0.051599,1.027398
5,45.302503,52.268129,55.020322,2.402228,0.000815,0.169989,0.167436,0.011726,3.153295
6,57.851302,72.345249,106.928270,7.038770,11.809640,9.061055,6.600767,0.072787,4.637877
7,4.801044,6.199524,25.429307,3.527693,6.366182,2.471396,6.761019,0.103493,0.194538
8,2.814477,3.266802,4.296112,0.306144,0.336748,0.210285,0.165335,0.010798,0.119475
9,4.397676,5.534110,10.466418,1.319835,2.080084,0.748104,0.775178,0.009107,0.213598


In [12]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

In [17]:
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()

# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask).multiply(deadWoodLitterRatio)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum()
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/WK1_Biome_Level_Statistics_with_Litter.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,279.026315,314.085872,343.041564,5.740267,0.092303,9.813583,12.921364,0.388174,8.008917
1,12.034080,15.036380,25.473836,2.594753,0.023949,2.370439,5.381522,0.066793,0.893290
2,4.214778,6.003045,7.496659,0.542121,0.036883,0.534293,0.375394,0.004924,0.610844
3,65.473775,75.575322,100.506556,6.016297,0.033124,7.158023,11.109822,0.613968,3.373905
4,32.450751,35.560268,39.345732,1.745028,0.053713,1.118143,0.799989,0.068590,1.366052
5,81.506726,94.040034,98.989825,4.322127,0.001367,0.305315,0.299936,0.021045,5.674159
6,70.579286,88.262046,130.454197,8.587408,14.408085,11.054750,8.053106,0.088803,5.658238
7,6.385594,8.245581,33.819262,4.691445,8.465855,3.286730,8.992014,0.137637,0.258764
8,3.434364,3.986294,5.242266,0.373552,0.410814,0.256609,0.201808,0.013187,0.145774
9,5.840211,7.348953,13.904134,1.753783,2.764777,0.994230,1.030285,0.012107,0.283661


In [57]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.